# Improting libraries

In [1]:

import numpy as np
import pandas as pd
import math 
from cvxopt import matrix, solvers
#from scipy.spatial import ConvexHull, convex_hull_plot_2d
#from scipy.optimize import line_search
#from datetime import datetime
from time import process_time
from scipy import linalg
from scipy.linalg import null_space 

To perform the experiments you first call the 'create_functions', then you will feed 'objective' and 'gradient' to the functions of the optimization algorithms. Gradient and objective will be to function where the regularization term is specified when create_functions is called. So when called inside the opt algorithms they just need xt. 

# Create Functions, Objective, gradient

In [2]:
def create_functions(Q, c, N, regularization=0, alpha1=0, alpha2=0): # I don't remeber why those default values for alpha1 and 2
    dim = N
    al1_up = 2 / (6 * (1 + 10 ** (-9)))               # the rules for al1 and al2 have been taken from the paper 
    al2_up = 2 / 25                                   # of the max clique 
    if alpha1 < 0 or alpha1 > al1_up:
        alpha1 = np.random.uniform(low=0, high=al1_up)
    if alpha2 < 0 or alpha2 > al2_up:
        alpha2 = np.random.uniform(low=0, high=al2_up)

    def objective(x):
        term = 0.0
        eps = (10 ** (-9)) * np.ones(x.shape)
        if regularization == 0:
            term = 0.0
        if regularization == 1:
            term = 1 / 2 * math.sqrt(np.sum(x * x))
        elif regularization == 2:
            term = alpha1 * np.sum((x + eps) ** 3)
        elif regularization == 3:
            term = alpha2 * np.sum(np.exp(-5 * x) - 1)
        obj = 1 / 2 * x.T @ (Q @ x) + term
        return obj

    def gradient(x): 
        gt = 0.0     
        eps = (10 ** (-9)) * np.ones(x.shape) 
        if regularization == 0:
            gt = 0.0
        if regularization == 1:
            gt = math.sqrt(np.sum(x))  ###########
        elif regularization == 2:
            gt = (alpha1 / 3) * np.sum((x + eps) ** 2)
        elif regularization == 3:
            gt = alpha2 * np.sum(np.exp(-5 * x))
        gr = Q @ x - gt
        return gr

    return objective, gradient

Below there are all the functions that are called inside the opt. alg. The arguments were specified in every function, but actually it is not necessary because every variable is declared inside the o. a. I leave these arguments so that each of those functions can be easily tested outside the function. But if everything works, you can consider to remove the arguments

In [3]:
#in the original code xt was defined like this, inside the class ConvexCombination
def value(self):
    return self.matrix.T @ self.weights # in our code this should be active_set.T @ weights

#xt = active_set.value()

#I don't know, maybe this can help you during the experiments 

# ConvexCombination

In [4]:
class ConvexCombination: ## This is the convex combination class for initializing the start_point_convcomb in the algorithms and also returning the caratheodory method
    ## this line initilizes the convex combination and accepts ConvexCombination(simplex[start_indexes], start_indexes, start_weights) which are from the simplex descriptor class
    ## Also start_indexes = np.arange(0, N) which N is the size of the vertices in the graph 
    ## w = np.random.rand(N),  This returns an array of random numbers of length N between 0 and 1
    ## start_weights = w / sum(w) which N is the size of vertices in the graph 
    def __init__(self, points_matrix, vertex_indexes, weights=False): 
        self.matrix = np.array(points_matrix) ## Making a numpy array out of points_matrix (simplex[start_indexes])
        self.indexes = np.array(vertex_indexes) ## Making a numpy array out of points_matrix (start_indexes)
        ## if we do not have the start_weights we initiate them like this 
        if (type(weights) == bool): ## if we have a type bool we can say that it is the defualt value 
            l = points_matrix.shape[0] ## .shape[0] is the size of N 
            self.weights = 1 / l * np.ones(l) ## all the weights are ((one / N(or L) )) in an array of size L which is N  
        else:
            self.weights = np.array(weights) ## Else the wights are the weights 

    def copy(self):
        return ConvexCombination(self.matrix, self.indexes, self.weights) # Intiate another object of class convexcombination

    def value(self):
        return self.matrix.T @ self.weights # we take the transpose of the matrix and the multiply it by the weights

    def drop(self, index):
        self.matrix = np.delete(self.matrix, index, 0)
        self.indexes = np.delete(self.indexes, index)
        self.weights = np.delete(self.weights, index)

    def size(self):
        return self.weights.shape[0]

# SimplexDescriptor

In [5]:
class SimplexDescriptor():
    def _get_column(self, k):
        r = np.zeros(self.dim)
        r[k] = 1
        return r

    def _change_column(self, r, i, k):
        r[i, k] = 1

    def __getitem__(self, key):
        if (type(key) == tuple):
            if (len(key) > 2):
                raise IndexError("Too many indexes: %d > 2" % len(key))
            k1, k2 = key
        else:
            k1, k2 = key, slice(None, None, None)
        if (type(k1) == int) or (type(k1) == np.int64):
            r = self._get_column(k1)
            return r[k2]
        if (type(k1) == slice):
            start, stop, step = k1.indices(self.vertices_numbers)
            rlen = (stop - start - 1) // step + 1
            indexes = list(range(start, stop, step))
        else:
            rlen = len(k1)
            indexes = k1
        i, r = 0, np.zeros([rlen, self.dim])
        for x in indexes:
            self._change_column(r, i, x)
            i += 1
        return r[:, k2]
        # return r

    def __init__(self, dimension):
        self.dim = dimension
        self.vertices_numbers = self.dim

    def __matmul__(self, v):
        if (len(v) != self.dim):
            raise IndexError('Wrong dimension')
        r = v[np.arange(0, self.dim)]
        return r

# Lipk and sz

In [6]:
def shrink_active_set(active_set, weights):
    zero_threshold = 1e-12
    w = weights
    l = weights.shape[0]
    zero_weights = np.where(np.abs(w) <= zero_threshold/l)
    if zero_weights[0].size:
        weights = np.delete(weights, weights[zero_weights])
        active_set = np.delete(active_set, active_set[zero_weights])
    return active_set, weights

def get_auxiliary_stepsize(Q, x_t, y_j, g, d, L=1e-2):
    if np.linalg.norm(y_j-(x_t + g / 2 / L)) < np.linalg.norm(g) / 2 / L and np.linalg.norm(y_j - x_t) < g@d/L:
        return gradient(y_j)@d/L/np.linalg.norm(d)
    else:
        return 0
# vertices are N= simplex
def LMO(simplex, gradient_xt): 
    sti = np.argmin(simplex @ gradient_xt) ## the gradient(xt)
    st = simplex[sti] # simplex[sti] only thing left to be defined is simplex
    return st, sti


def lipk_and_sz(Q, x_t, f_x, g, gamma_max, d_t, x0, L):
    
    if type(x0) == int:
        x0 = x_t 
        ##print("x0:", x0)
    epsilon = 10e-3
    k1 = 0.9
    k2 = 2
    nL = k1 * L
    clip = g ** 2 / (2 * (objective(x0) - f_x) * d_t.T @ d_t)
    ##print("print clip: ", clip) ## clip should not be zero
    M = np.clip(clip ,nL, L) ## the function clip is bounded by upper bound nL and lower bound L
    ##print("M:", M)
    gamma = min(g / (M * d_t.T @ d_t), gamma_max)
    ##print("gamma:", gamma)
    while objective(x_t + (gamma * d_t)) > (f_x - (gamma * g) + (g * (gamma ** 2) * M) / (d_t.T @ d_t)) :
        M = k2 * M
        ##print("Updating M:", M)
        gamma = min(g/(M*d_t.T@d_t), gamma_max)
    return gamma, M


# FW Away step with Lipschiz Step

In [7]:
#gradient, objective = create_functions(Q, c, N)
# I think you can remove fx and gradient as arguments
def FW_AS_our(objective,  gradient, Q, N, active_set = None, max_iter = 1000):
    FW_AS_our.label = FW_AS_our
    eps = 1e-6
    xt  = active_set.value()
    weights = active_set.weights
    #print("Do we get here?")
    for i in range(max_iter):
        if i==0:
            x0 = 1
            L = 1
        fxt = objective(xt)
        #print(" I wonder if we get here")  ## We do 
        st, sti = LMO(N, gradient(xt)) #defined
        #print(" I wonder if we get here")  ## We do 
        dfw = st - xt
        vt = np.max(active_set.matrix @gradient(xt)) # feed to the function the proprer value for active set 
        vti = np.argmax(active_set.matrix @gradient(xt)) # gradient is a fucntion itself
        daw = xt - vt
        gfw =  - gradient(xt)@dfw
        #print(" I wonder if we get here")  ## We do 
        if  gfw  <= eps: #check the sign of the gradient (or the sign of the gap)
            return [xt, active_set, fxt, "FW_AS"]
        if gfw >= gradient(xt) @ daw:
            gamma_max=1
            #print(" I wonder if we get here")
            ##Q, x_t, f_x, g, gamma_max, d_t, x0=0, L=0
            gamma, L = lipk_and_sz(Q, xt, fxt, gfw, gamma_max, dfw, x0, L) # we should change L=1 with L and x0=[] with x0
            #print("How about HERE")
            x0 = xt # this is required by the the function lipk_and_sz, because at every call of this function we need to feed xt and xt-1
            xt += gamma * dfw
            weights = (1-gamma) * weights
            weights[sti] += gamma #problem? #this shouldn't be a problem because weights'd be a matrix and in this way you'd update one row
            if (gamma == 1):
                active_set = st
                weights = weights[sti]
              
            else:
              active_set.matrix = np.vstack((active_set.matrix, st))
              active_set.weights = np.hstack((active_set.weights, gamma))
          
        else:
            weight_v_t = weights[vti]
            gamma_max = weight_v_t / (1 - weight_v_t) # in this case the weights should not be one. if the weights are equal to one then gamma-max is 
            ## equal to one as well
            ## def lipk_and_sz(Q, x_t, f_x, g, gamma_max, d_t, x0, L):
            gamma, L = lipk_and_sz(Q, xt, fx, gaw, gamma_max, daw,  x0, L) # we should change L=0 with L and x0=0 with x0
            x0 = xt # this is required by the the function lipk_and_sz, because at every call of this function we need to feed xt and xt-1
            xt += gamma*daw
            weights = (1+gamma) * weights
            weights[vti] -= gamma #problem? #this shouldn't be a problem because weights'd be a matrix and in this way you'd update one row
        if gamma == gamma_max:
            active_set = np.delete(active_set, active_set[vti], 0) #active_set.drop(vt)
            weights = np.delete(weights, weight_v_t)
    shrink_active_set(active_set, weights) #define again 
    return [xt, active_set, fxt, "FW_AS"] 



# Data Set

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
dimacs_dir_path = "/content/drive/MyDrive/Dimacs/dimacs/"
to_be_tested = "/content/drive/MyDrive/Dimacs/dimacs/to_be_tested"

# Check Clique

In [10]:

def check_clique(Q, x):
    print(x)
    findx, = np.where(abs(x)>1e-3)
    print(findx)
    c = findx.size
    v = abs(sum(sum(Q[findx][:, findx])) + c*(2*(c-1)+1))
    print("Check clique:", c, v)
    return v<0.1, c


# Testing Part

In [ ]:
test_results = {}
algo_list = []
#objective,  gradient, xt, Q, N, weights = np.array([]), active_set = None, max_iter = 1000
options = {"verbosity": 0, "epsilon": 1e-6}
# algo_list.append(FW_AwayStep_ClassicLipschizStep_Algorithm(**options))
# algo_list.append(FW_Pairwise_ClassicLipschizStep_Algorithm(**options))
algo_list.append(FW_AS_our)  #####################
#algo_list.append(FW_Pairwise_ClassicArmijoStep_Algorithm(**options))  ####################
#for A in algo_list:
#    A.set_line_search_parameters()
#algo_list.append(FW_AwayStep_SSC_Algorithm(**options))
#algo_list.append(FW_Pairwise_SSC_Algorithm(**options))
# =============================================================
#algo_names = []
#for algo in algo_list:
#    algo_names.append(algo.label)
# =============================================================
np.set_printoptions(threshold=100)

# ================================================================
# This part is for preprocessing the graphs given to matrices 
def load_dimacs_graph(name):
    t0 = process_time()
    f = open(dimacs_dir_path + name, 'r') 
    edges = []
    N, M, m, n = 0, 0, 0, 0
    t1 = process_time()
    rows = f.readlines()
    t2 = process_time()
    for row in rows:
        args = row.split()
        if (not N):
            # if (match):
            if (len(args) >= 4) and (args[0] == "p"):
                # n, m = int(match.group(1)), int(match.group(2))
                n, m = int(args[2]), int(args[3])
                N = max(N, n)
                M = m
        if (len(args) >= 3) and (args[0] == "e"):
            # v1, v2 = int(match.group(1)), int(match.group(2))
            v1, v2 = int(args[1]), int(args[2])
            edges.append((v1, v2))
            N = max(N, v1, v2)
    print("Caricamento %s: Controllo caricamento" % name, (n == N), (m == M), (M == len(edges)))
    t3 = process_time()
    Q = -np.eye(N)
    for v1, v2 in edges:
        Q[v1 - 1, v2 - 1] = -2
        Q[v2 - 1, v1 - 1] = -2
    del row, edges
    c = np.zeros(N)
    t4 = process_time()
    return Q, c, N, M, (t0, t1, t2, t3, t4)

"""
# ====================================================================================================================

def check_clique(Q, x):
    findx, = np.where(abs(x) > 1e-3)
    print(findx)
    c = findx.size
    v = abs(sum(sum(Q[findx][:, findx])) + c * (2 * (c - 1) + 1))
    print("Check clique:", c, v)
    return v < 0.1, c

"""

"""
# This part is where we define objective functions and gradients 
# If we want to use this optimization algorithms for other continuous optimization problems we have to change this part 
def create_functions(Q, c, N, regularization=0, alpha1=0, alpha2=0): # I don't remeber why those default values for alpha1 and 2
    dim = N
    al1_up = 2 / (6 * (1 + 10 ** (-9)))               # the rules for al1 and al2 have been taken from the paper 
    al2_up = 2 / 25                                   # of the max clique 
    if alpha1 < 0 or alpha1 > al1_up:
        alpha1 = np.random.uniform(low=0, high=al1_up)
    if alpha2 < 0 or alpha2 > al2_up:
        alpha2 = np.random.uniform(low=0, high=al2_up)

    def objective(x):
        term = 0.0
        eps = (10 ** (-9)) * np.ones(x.shape)
        if regularization == 0:
            term = 0.0
        if regularization == 1:
            term = 1 / 2 * math.sqrt(np.sum(x * x))
        elif regularization == 2:
            term = alpha1 * np.sum((x + eps) ** 3)
        elif regularization == 3:
            term = alpha2 * np.sum(np.exp(-5 * x) - 1)
        obj = 1 / 2 * x.T @ (Q @ x) + term
        return obj

    def gradient(x): 
        gt = 0.0     
        eps = (10 ** (-9)) * np.ones(x.shape) 
        if regularization == 0:
            gt = 0.0
        if regularization == 1:
            gt = math.sqrt(np.sum(x))  ###########
        elif regularization == 2:
            gt = (alpha1 / 3) * np.sum((x + eps) ** 2)
        elif regularization == 3:
            gt = alpha2 * np.sum(np.exp(-5 * x))
        gr = Q @ x - gt
        return gr

    return objective, gradient
"""

reg_list = ["phiB", "phi1", "phi2"]
# ==================================================
# This part of the code is for testing
test_name_list_file = open(to_be_tested)
test_name_list = test_name_list_file.read().split()
test_name_list_file.close()
print("Test list: ", test_name_list[:3])
test_results = {}
test_results_reg = {}
error_list = []
times = {ALG: [] for ALG in algo_list}
clique_nums = {ALG: [] for ALG in algo_list}
for test_name in test_name_list[:3]:  # 
    if (test_name[0] == "#"):
        continue
    print("*" * 30 + "\nRunning test %s\n" % test_name + "*" * 30)
    Q, c, N, M, times = load_dimacs_graph(test_name + ".clq")
    simplex = SimplexDescriptor(N) # Ke betavanim baadan parameter be algorithm ezafe konim
    # ==============================================================================
    for reg in range(1, 3):  # for faster runs change to range(1,3)
        print("regularization_term: ", reg_list[reg])
        objective, gradient = create_functions(Q, c, N, regularization=reg + 1)
        # ===================================================================================
        times = {ALG: [] for ALG in algo_list}
        clique_nums = {ALG: [] for ALG in algo_list}
        iteration_nums = {ALG: [] for ALG in algo_list}
        for i in range(0, 10):
            print("===\nExecution #%02d\n===" % i)
            np.random.seed(i)
            w = np.random.rand(N)
            start_weights = w / sum(w)
            start_indexes = np.arange(0, N)
            ##==========================================================
            for ALG in algo_list:
                #ALG.set_parameters(N, simplex)
                #ALG.set_objective(objective, gradient, start_point=simplex[N - 1], alpha=1)  # alpha = 2 * LipschitzDFLowerBound)
                x0 = ConvexCombination(simplex[start_indexes], start_indexes, start_weights)
                t0 = process_time()
                #ALG.run(max_iter=10000, start_point_convcomb=x0)
                #objective,  gradient, Q, N, active_set = None, max_iter = 1000
                x, cv, fx, status = ALG(objective, gradient, Q, simplex, x0, max_iter = 10000)
                t1 = process_time()
                delta_t = t1 - t0
                #ALG.result
                #iterations = ALG.t
                check, cliq_num = check_clique(Q, x)
                if (check):
                    print("Is correct")
                else:
                    print("Is NOT correct")
                    error_list.append((test_name, i, x, check, cliq_num))
                print(x, fx, "\n")
                times[ALG].append(delta_t)
                clique_nums[ALG].append(cliq_num)
                #iteration_nums[ALG].append(iterations)
            ##==========================================================
        for ALG in algo_list:
            print(ALG)
            print(clique_nums[ALG], sum(clique_nums[ALG]) / 10)
            print(times[ALG], sum(times[ALG]) / 10)
            print(iteration_nums[ALG], sum(iteration_nums[ALG]) / 10)
        test_results[test_name] = {"cpu_time": times, "clique_number": clique_nums, "iteration_number": iteration_nums}
        test_results_reg[reg_list[reg]] = test_results

# output_to_json()
# ==================================================
# This is the part we run to show different cpu times in different algorithms and compare results 
table_latex = ""
test_table = []
test_table_better = []
test_table_better_reg = []
test_names = []
for test_name in test_results:
    for regular in test_results_reg:
        print("test_name: ", test_name)
        print(regular)
        result = test_results[test_name]
        data_to_show_str = [test_name.replace("_", "\\_")]
        data_to_show = []  # test_name]
        data_to_show_better = []
        test_name_reg = test_name + " " + regular
        test_names.append(test_name_reg)
        for alg_label in result["clique_number"]:
            p = 4
            d = []
            clique_number = np.array(result["clique_number"][alg_label])
            cpu_time = np.array(result["cpu_time"][alg_label])
            v = [
                min(clique_number),
                np.mean(clique_number),
                max(clique_number),
                np.std(clique_number),
                min(cpu_time),
                np.mean(cpu_time),
                max(cpu_time),
                np.std(cpu_time)
            ]
            vbetter = [min(clique_number),
                       np.mean(clique_number),
                       max(clique_number),
                       np.std(clique_number),
                       np.mean(cpu_time)
                       ]
            d[0:2] = [str(v[0]), "%.1f" % v[1], str(v[2])]
            d[3:8] = ["%.2f" % l for l in v[3:8]]
            data_to_show_better += vbetter
            data_to_show += v
            data_to_show_str += d
        # print(v, d)
        test_table_better.append(data_to_show_better)
        test_table.append(data_to_show)
        table_latex += " & ".join(data_to_show_str) + " \\\\\n"
    # test_table_better_reg.append
print(len(test_table_better))
clmsbetter = ['min clique number', 'mean clique number', 'max clique number', 'std clique number',
              'mean cpu time']
print(test_results_reg)
muxbetter = pd.MultiIndex.from_product([algo_names, clmsbetter])
df = pd.DataFrame(test_table_better, index=test_names, columns=muxbetter)
with open('test_reg.txt', 'w') as f:
    f.write(df.to_string())
display(df)


Test list:  ['brock200_2', 'brock200_4', 'brock400_2']
******************************
Running test brock200_2
******************************
Caricamento brock200_2.clq: Controllo caricamento True True True
regularization_term:  phi1
===
Execution #00
===


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in double_scalars


[3.49920115e-05 4.56000342e-05 3.84318222e-05 ... 1.62176129e-05
 3.69990358e-06 1.10260068e-01]
[  9  31  40  48  68  69  91 133 199]
Check clique: 9 0.0
Is correct
[3.49920115e-05 4.56000342e-05 3.84318222e-05 ... 1.62176129e-05
 3.69990358e-06 1.10260068e-01] -0.939347557459073 

===
Execution #01
===
[2.81347428e-05 4.85973021e-05 7.71639393e-09 ... 1.58114367e-05
 4.16114743e-05 6.40261899e-05]
[  9  45  69  88 105 135 157 160]
Check clique: 8 0.0
Is correct
[2.81347428e-05 4.85973021e-05 7.71639393e-09 ... 1.58114367e-05
 4.16114743e-05 6.40261899e-05] -0.9325294084696105 

===
Execution #02
===
[3.04967505e-05 1.81347493e-06 3.84475125e-05 ... 1.23869223e-01
 6.41218891e-05 4.91216232e-05]
[  9  40  69 105 160 184 196 197]
Check clique: 8 0.0
Is correct
[3.04967505e-05 1.81347493e-06 3.84475125e-05 ... 1.23869223e-01
 6.41218891e-05 4.91216232e-05] -0.9325262350270395 

===
Execution #03
===
[3.64411984e-05 4.68515859e-05 1.92464736e-05 ... 4.12045668e-06
 5.85020699e-05 2.94951